In [2]:
import sys

import pandas as pd

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sem_covid import config
import json
import numpy as np
import re
from langdetect import detect, detect_langs, DetectorFactory

DetectorFactory.seed = 0

from sem_covid.services.store_registry import StoreRegistry, store_registry



In [3]:
es_store = store_registry.es_index_store()



In [4]:
# create model for document embeddings
from sem_covid.services.model_registry import embedding_registry

emb_model = embedding_registry.sent2vec_universal_sent_encoding()


INFO:absl:Using /tmp/tfhub_modules to cache modules.


In [5]:
# create model for topic embeddings
import mlflow

EXPERIMENT_ID = '101'
BUCKET_NAME = 'mlflow'

best_run = mlflow.search_runs(
    experiment_ids=EXPERIMENT_ID,
    order_by=['parameters.freq_topic_minus_1 ASC']
).iloc[0]



bert_model = store_registry.minio_feature_store(BUCKET_NAME).get_features(
    features_name=f'{EXPERIMENT_ID}/{best_run.run_id}/artifacts/model/model.pkl')



In [6]:
def make_target_group_l1_column(dataset: pd.DataFrame):
    for col in TARGET_GROUPS_L1:
        dataset[col]=dataset[col].apply(lambda x: col if x == 1 else "" )

    dataset["pwdb_target_group_l1"] = dataset[TARGET_GROUPS_L1].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)\
        .apply(lambda x: x.split())


In [7]:
def make_document_embeddings_column(dataset: pd.DataFrame):
    dataset["document_embeddings"] = emb_model.encode(dataset.content.values)


In [25]:
def make_topic_embeddings_column(dataset: pd.DataFrame):
    bert_probabilities = bert_model.transform(documents=dataset['content'],
                                              embeddings=np.array(dataset["document_embeddings"].values.tolist()).astype("float32"))[1]
    dataset['topic_embeddings'] = bert_probabilities.tolist()

In [9]:
def create_new_column_with_defined_value(dataset: pd.DataFrame, column_name: str, value=None, empty_array=False):
    if empty_array:
       dataset[column_name] = [np.empty(0,dtype=float)]*len(dataset)
    else:
        dataset[column_name] = value


In [12]:
TARGET_GROUPS_L1 = ["businesses", "workers", "citizens"]
COMMON_DATASET_COLUMNS = ["title","content","date","doc_source","country","pwdb_category",
                          "pwdb_target_group_l1","pwdb_funding", "pwdb_type_of_measure",
                          "pwdb_actors", "document_embeddings", "topic_embeddings"]

SPECIFIC_DATASET_COLUMNS = ["eu_cellar_subject_matter_labels", "eu_cellar_resource_type_labels","eu_cellar_directory_code_labels",
                            "eu_cellar_author_labels", "pwdb_target_group_l2", "ireland_keyword", "ireland_department_data",
                            "ireland_campaign", "ireland_page_type", "eu_timeline_topic"]

In [311]:
#Load dataframes from elastic
pwdb_df = es_store.get_dataframe(index_name=config.PWDB_ELASTIC_SEARCH_INDEX_NAME)
eu_cellar_df = es_store.get_dataframe(index_name=config.EU_CELLAR_ELASTIC_SEARCH_INDEX_NAME + "_enriched")
eu_timeline_df = es_store.get_dataframe(index_name=config.EU_TIMELINE_ELASTIC_SEARCH_INDEX_NAME + "_enriched")
ir_timeline_df = es_store.get_dataframe(index_name=config.IRELAND_TIMELINE_ELASTIC_SEARCH_INDEX_NAME + "_enriched")



100% (1381 of 1381) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (3175 of 3175) |####################| Elapsed Time: 0:00:01 Time:  0:00:01
100% (231 of 231) |######################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1921 of 1921) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [19]:
#textual columns in PWDB dataset into one content column
textual_columns = [col for col in pwdb_df.columns if "description" in col]
textual_columns.insert(0,"title")
pwdb_df["content"] = pwdb_df[textual_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
pwdb_df["content"]

_id
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b    Temporary layoff scheme and reimbursement of r...
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d    Protocol for the containment of the COVID-19 s...
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2    ELY centres' business development aid to micro...
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33    Regularisation of immigrants' presence on nati...
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa    Economic reactivation measures to face the imp...
                                                                                               ...                        
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f    Government support to assist the sports sector...
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6    National public sector collective agreement 20...
tika/3df93a8

In [20]:
# Create necessary columns
make_target_group_l1_column(pwdb_df)
make_document_embeddings_column(pwdb_df)
make_topic_embeddings_column(pwdb_df)

create_new_column_with_defined_value(pwdb_df, "doc_source", "ds_pwdb")

# create specific dataset columns

for column in SPECIFIC_DATASET_COLUMNS:
    create_new_column_with_defined_value(pwdb_df,column_name=column,empty_array=True)



In [21]:
#add values if the specific column exists in the current dataset
pwdb_df["pwdb_target_group_l2"] = pwdb_df["target_groups"]



In [22]:
# rename columns in the current dataset and prepare new version for merging
pwdb_df.rename(columns={"category": "pwdb_category", "funding": "pwdb_funding", "type_of_measure": "pwdb_type_of_measure"
    , "actors": "pwdb_actors", "start_date": "date"}, inplace=True)

tmp_pwdb_df = pd.DataFrame(pwdb_df[COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS])

In [23]:
# Verify if the new version has all the necessary columns
if tmp_pwdb_df.columns.tolist() == COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS:
    print("ready for merge")
else:
    raise ValueError("Some columns are missing!! Not ready for merge")

ready for merge


In [ ]:
# tmp_pwdb_df["document_embeddings"]
#
#
#
#
# import mlflow
#
# EXPERIMENT_ID = '101'
# BUCKET_NAME = 'mlflow'
#
# best_run = mlflow.search_runs(
#     experiment_ids=EXPERIMENT_ID,
#     order_by=['parameters.freq_topic_minus_1 ASC']
# ).iloc[0]
#
#
#
# bert_model = store_registry.minio_feature_store(BUCKET_NAME).get_features(
#     features_name=f'{EXPERIMENT_ID}/{best_run.run_id}/artifacts/model/model.pkl')
#
#
#
# tmp_pwdb_df["document_embeddings"]
#
# # type(tmp_pwdb_df["document_embeddings"][0])
#
#
# bert_probabilities = bert_model.transform(documents=tmp_pwdb_df['content'], embeddings=np.array(tmp_pwdb_df["document_embeddings"].values.tolist()).astype("float32"))[1]
# tmp_pwdb_df['topic_based_embedding'] = bert_probabilities.tolist()
#
#
#
#
# tmp_pwdb_df['topic_based_embedding']
#
#
#
# np.array(tmp_pwdb_df["document_embeddings"].values.tolist())[0]
#
#
#
#
# set(map(len,tmp_pwdb_df['document_embeddings'].values.tolist()))
#
#
#
# set(tmp_pwdb_df["content"].apply(lambda x: len(x) < 10))


In [ ]:
# tmp_pwdb_df["document_embeddings"] = emb_model.encode(tmp_pwdb_df.content.values)
# tmp_pwdb_df["document_embeddings"]
#


In [ ]:
# # Create new dataframe with the wanted content and columns from PWDB dataset
# tmp_pwdb_df = pd.DataFrame(pwdb_df[["creation_date", "title", "merged_content"]])
# tmp_pwdb_df["doc_source"] = "pwdb"


# # Print only textual columns
# pwdb_df[textual_columns]
#
# #Merge content of the textual columns into one new column
# pwdb_df["merged_content"] = pwdb_df[textual_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
# pwdb_df["merged_content"]
#
# #What are the columns that shows dates
# date_columns = [col for col in pwdb_df.columns if "date" in col]
# pwdb_df[date_columns]
#
# # Create new dataframe with the wanted content and columns from PWDB dataset
# tmp_pwdb_df = pd.DataFrame(pwdb_df[["creation_date", "title", "merged_content"]])
# tmp_pwdb_df["doc_source"] = "pwdb"
#
# # See the newly created PWDB dataframe
# tmp_pwdb_df

In [127]:
# Print dataframe columns
eu_cellar_df.columns



Index(['work', 'title', 'cdm_types', 'cdm_type_labels', 'resource_types',
       'resource_type_labels', 'eurovoc_concepts', 'eurovoc_concept_labels',
       'subject_matters', 'subject_matter_labels', 'directory_codes',
       'directory_codes_labels', 'celex_numbers', 'legal_elis', 'id_documents',
       'same_as_uris', 'authors', 'author_labels', 'full_ojs', 'oj_sectors',
       'internal_comments', 'is_in_force', 'dates_document', 'dates_created',
       'legal_dates_entry_into_force', 'legal_dates_signature', 'manifs_pdf',
       'manifs_html', 'pdfs_to_download', 'htmls_to_download', 'dossiers',
       'related_works', 'work_sequences', 'eu_cellar_core',
       'eu_cellar_extended', 'metadata', 'content_path', 'content', 'language',
       'businesses', 'citizens', 'workers', 'category', 'subcategory',
       'type_of_measure', 'funding'],
      dtype='object')

In [81]:
# eu_cellar_df[["title", "content"]].agg(lambda x: " ".join([item if item else "" for item in x]), axis=1)

_id
0       Prior notification of a concentration (Case M....
1       COMMISSION STAFF WORKING DOCUMENT […] Accompan...
2       P9_TA(2020)0157 Amending Regulations (EU) No 5...
3       Opinion No 6/2020 (pursuant to Article 287(4) ...
4       Council Recommendation (EU) 2021/816 20 May 20...
                              ...                        
3170    Case C-774/19: Judgment of the Court (Sixth Ch...
3171    Calls for proposals and related activities und...
3172                                                     
3173    European Parliament resolution of 18 June 2020...
3174    Proposal for a COUNCIL DECISION on the positio...
Length: 3175, dtype: object

In [129]:
# create columns
create_new_column_with_defined_value(eu_cellar_df,"doc_source", "ds_eu_cellar")
create_new_column_with_defined_value(eu_cellar_df,"country", "European Union")
create_new_column_with_defined_value(eu_cellar_df,"pwdb_actors", "EU (Council, EC, EP)")
make_target_group_l1_column(eu_cellar_df)
eu_cellar_df["date"] = eu_cellar_df["dates_document"]
eu_cellar_df["content"] = eu_cellar_df["title"].replace({None: ""}) + eu_cellar_df["content"].replace({None: ""})
eu_cellar_df["content"] = eu_cellar_df["content"].replace({"":None})
eu_cellar_df.dropna(subset=["content"], inplace=True)

make_document_embeddings_column(eu_cellar_df)
make_topic_embeddings_column(eu_cellar_df)
# create specific dataset columns

for column in SPECIFIC_DATASET_COLUMNS:
    create_new_column_with_defined_value(eu_cellar_df,column_name=column,empty_array=True)

eu_cellar_df[SPECIFIC_DATASET_COLUMNS]



,eu_cellar_subject_matter_labels,eu_cellar_resource_type_labels,eu_cellar_directory_code_labels,eu_cellar_author_labels,pwdb_target_group_l2,ireland_keyword,ireland_department_data,ireland_campaign,ireland_page_type,eu_timeline_topic
_id,,,,,,,,,,
0,[],[],[],[],[],[],[],[],[],[]
1,[],[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],[],[]
3,[],[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...
3168,[],[],[],[],[],[],[],[],[],[]
3170,[],[],[],[],[],[],[],[],[],[]
3171,[],[],[],[],[],[],[],[],[],[]


In [130]:
#add values if the specific column exists in the current dataset
eu_cellar_specific_columns = ["subject_matter_labels",
                              "resource_type_labels",
                              "directory_codes_labels",
                              "author_labels"]
for column_name in eu_cellar_specific_columns:
    eu_cellar_df["eu_cellar_" + column_name] = eu_cellar_df[column_name]
    eu_cellar_df["eu_cellar_" + column_name] = eu_cellar_df["eu_cellar_" + column_name].apply(lambda x: x if x else [])


In [131]:
# rename columns in the current dataset and prepare new version for merging
eu_cellar_df.rename(columns={"category": "pwdb_category", "funding": "pwdb_funding", "type_of_measure": "pwdb_type_of_measure"
    }, inplace=True)


In [132]:
tmp_eu_cellar_df = pd.DataFrame(eu_cellar_df[COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS])


In [133]:
# Verify if the new version has all the necessary columns
if tmp_eu_cellar_df.columns.tolist() == COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS:
    print("ready for merge")
else:
    raise ValueError("Some columns are missing!! Not ready for merge")



ready for merge


In [134]:
# Print dataframe columns
eu_timeline_df.columns

Index(['month_name', 'date', 'title', 'abstract', 'presscorner_links',
       'all_links', 'detail_link', 'detail_type', 'detail_date',
       'detail_location', 'detail_content', 'detail_title', 'detail_pdf_link',
       'press_contacts', 'topics', 'for_more_information_links', 'businesses',
       'citizens', 'workers', 'category', 'subcategory', 'type_of_measure',
       'funding'],
      dtype='object')

In [239]:
# create columns
CONTENT_COLUMNS = ["title", "abstract", "detail_content"]
eu_timeline_df["content"] = eu_timeline_df[CONTENT_COLUMNS].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
create_new_column_with_defined_value(eu_timeline_df,"doc_source", "ds_eu_timeline")
create_new_column_with_defined_value(eu_timeline_df,"country", "European Union")
create_new_column_with_defined_value(eu_timeline_df,"pwdb_actors", "EU (Council, EC, EP)")
make_target_group_l1_column(eu_timeline_df)

make_document_embeddings_column(eu_timeline_df)
# make_topic_embeddings_column(eu_timeline_df)
# create specific dataset columns

for column in SPECIFIC_DATASET_COLUMNS:
    create_new_column_with_defined_value(eu_timeline_df,column_name=column,empty_array=True)

eu_timeline_df[SPECIFIC_DATASET_COLUMNS]

,eu_cellar_subject_matter_labels,eu_cellar_resource_type_labels,eu_cellar_directory_code_labels,eu_cellar_author_labels,pwdb_target_group_l2,ireland_keyword,ireland_department_data,ireland_campaign,ireland_page_type,eu_timeline_topic
_id,,,,,,,,,,
0,[],[],[],[],[],[],[],[],[],[]
1,[],[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],[],[]
3,[],[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...
205,[],[],[],[],[],[],[],[],[],[]
206,[],[],[],[],[],[],[],[],[],[]
207,[],[],[],[],[],[],[],[],[],[]


In [225]:
eu_timeline_df["content"] = eu_timeline_df["title"].replace({None: ""}) + \
                            eu_timeline_df["abstract"].replace({None: ""}) + \
                            eu_timeline_df["detail_content"].replace({None: ""})

In [259]:
tmp_df_timeline = pd.DataFrame(eu_timeline_df.head(50).copy())

In [277]:
for x in tmp_df_timeline["content"].values:
    print(x)
    print("________________________")

EU countries adopt guidelines on proof of vaccination for medical purposesThe EU countries, with the support of the European Commission, adopted  . These guidelines aim to support the interoperability of vaccination certificates, meaning the contents of the vaccination certificates is uniform, and establish a minimum dataset for each certificate.
________________________
Commission endorses Latvia's €1.8 billion recovery and resilience planThe Commission adopted a   of Latvia's recovery and resilience plan, an important step towards the EU disbursing €1.8 billion in grants under the Recovery and Resilience Facility. It will play a key role in enabling Latvia emerge stronger from the COVID-19 pandemic.The European Commission has today adopted a positive assessment of Latvia's recovery and resilience plan. This is an important step towards the EU disbursing 1.8 billion in grants under the Recovery and Resilience Facility (RRF). This financing will support the implementation of the crucia

In [312]:
tmp_df_timeline = pd.DataFrame(eu_timeline_df.head(80).copy())


In [313]:
tmp_df_timeline["content"] = tmp_df_timeline["title"].replace({None: ""}) + \
                            tmp_df_timeline["abstract"].replace({None: ""}) + \
                            tmp_df_timeline["detail_content"].replace({None: ""})


In [320]:
tmp_df_timeline["detail_content"][tmp_df_timeline["detail_content"]== None]

Series([], Name: detail_content, dtype: object)

In [314]:
make_document_embeddings_column(tmp_df_timeline)



In [316]:

bert_probabilities = bert_model.transform(documents=tmp_df_timeline["title"].replace({None: ""}),
                                          embeddings=np.array(list(tmp_df_timeline["document_embeddings"])))[1]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [278]:
papa = np.array(tmp_df_timeline.document_embeddings.values.tolist()).flatten()

In [280]:
[x for x in papa if x == ]

[]

In [233]:
make_topic_embeddings_column(tmp_df_timeline)

ZeroDivisionError: division by zero

In [206]:
make_document_embeddings_column(eu_timeline_df)
make_topic_embeddings_column(eu_timeline_df)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [163]:
CONTENT_COLUMNS = ["title", "abstract", "detail_content"]
eu_timeline_df["content"] = eu_timeline_df[CONTENT_COLUMNS].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)



for x in eu_timeline_df["content"].values:
    if x == "":
        print("empty")

eu_timeline_df[eu_timeline_df.content == np.NaN]


p = np.array(eu_timeline_df.document_embeddings.values.tolist()).flatten()




[x for x in p if x == np.NaN]

In [201]:
eu_timeline_df["content"] = eu_timeline_df[CONTENT_COLUMNS].agg(lambda x: " ".join(item if item else "" for item in x), axis=1)

In [197]:
eu_timeline_df["content"]

_id
0      EU countries adopt guidelines on proof of vacc...
1      Commission endorses Latvia's €1.8 billion reco...
2      Commission unveils EU vaccines strategy The Co...
3      European roadmap shows path towards common lif...
4      Commission puts forward rules on rapid antigen...
                             ...                        
205    Commission identifies five promising candidate...
206    Setting out coordinated response to counter th...
207    Prolonging State aid Temporary Framework to fu...
208    Commission and EIB provide CureVac with a €75 ...
209    Commission endorses France's €39.4 billion rec...
Name: content, Length: 210, dtype: object

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [240]:
#add values if the specific column exists in the current dataset
eu_timeline_df["eu_timeline_topic"] = eu_timeline_df["topics"]

In [241]:
# rename columns in the current dataset and prepare new version for merging
eu_timeline_df.rename(columns={"category": "pwdb_category", "funding": "pwdb_funding", "type_of_measure": "pwdb_type_of_measure"
    }, inplace=True)


In [242]:

tmp_eu_timeline_df = pd.DataFrame(eu_timeline_df[COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS])


KeyError: "['topic_embeddings'] not in index"

In [243]:
# Verify if the new version has all the necessary columns
if tmp_eu_timeline_df.columns.tolist() == COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS:
    print("ready for merge")
else:
    raise ValueError("Some columns are missing!! Not ready for merge")


NameError: name 'tmp_eu_timeline_df' is not defined

In [16]:
#What are the columns that shows dates
date_columns = [col for col in eu_timeline_df.columns if "date" in col]
eu_timeline_df[date_columns]


,date,detail_date
_id,,
01dc34ca5f6bfa5316012d6f89a4256b8d2ce6e3a12210b0406903dd15bf36ac,2021-01-28,NaN
04d776c061243576ffc1e1c5a724d41ecca2ed12339b2ff473c5aa2ff1a071c7,2021-06-22,2021-06-22
04eb55005a803c7353e5d5ed339d7f3a72cc8da6e49f2b8b152bf8fb40a1398f,2020-06-16,2020-06-17
073e22ec2b44a5f2f6289f09dc954ecd6ca6cbb4a4b33ffd88c3c150572569c0,2020-04-15,2020-04-15
0a3650bef3282d09d971dd92dc01570b7ecbdb67a4bdd0ff1c8cf8443a06b937,2020-12-18,NaN
...,...,...
fa793c764aaf89c283f747036c9d73fd66f0a7e3692be56b8ac985610bef2417,2021-06-29,2021-06-29
fb9c4d1634da94c73fd529b44ec40057c28a5c925a8b8e71d73c49e5a815784d,2020-03-13,2020-03-13
fbad1d421cf80d3e0e4c6b6d3d60a2a6ec8bed4ee19e07cd15d963fa74352420,2021-01-28,2020-03-19


In [44]:
eu_timeline_df["detail_content"]

_id
01dc34ca5f6bfa5316012d6f89a4256b8d2ce6e3a12210b0406903dd15bf36ac                                                  NaN
04d776c061243576ffc1e1c5a724d41ecca2ed12339b2ff473c5aa2ff1a071c7    The European Commission has today adopted a po...
04eb55005a803c7353e5d5ed339d7f3a72cc8da6e49f2b8b152bf8fb40a1398f    Today, to help protect people everywhere, the ...
073e22ec2b44a5f2f6289f09dc954ecd6ca6cbb4a4b33ffd88c3c150572569c0    Today, the Commission, in cooperation with the...
0a3650bef3282d09d971dd92dc01570b7ecbdb67a4bdd0ff1c8cf8443a06b937                                                  NaN
                                                                                          ...                        
fa793c764aaf89c283f747036c9d73fd66f0a7e3692be56b8ac985610bef2417    The EU Strategy on COVID-19 Therapeutics deliv...
fb9c4d1634da94c73fd529b44ec40057c28a5c925a8b8e71d73c49e5a815784d    COVID-19 is a severe public health emergency f...
fbad1d421cf80d3e0e4c6b6d3d60a2a6ec8bed4ee19e07cd15d9

In [17]:
# Create new dataframe with the wanted content and columns from EU timeline dataset
tmp_eu_timeline_df = pd.DataFrame(eu_timeline_df[["date", "title", "detail_content"]])
tmp_eu_timeline_df["doc_source"] = "eu_timeline"

In [18]:
# See the newly created EU timeline  dataframe
tmp_eu_timeline_df

,date,title,detail_content,doc_source
_id,,,,
01dc34ca5f6bfa5316012d6f89a4256b8d2ce6e3a12210b0406903dd15bf36ac,2021-01-28,EU countries adopt guidelines on proof of vacc...,NaN,eu_timeline
04d776c061243576ffc1e1c5a724d41ecca2ed12339b2ff473c5aa2ff1a071c7,2021-06-22,Commission endorses Latvia's €1.8 billion reco...,The European Commission has today adopted a po...,eu_timeline
04eb55005a803c7353e5d5ed339d7f3a72cc8da6e49f2b8b152bf8fb40a1398f,2020-06-16,Commission unveils EU vaccines strategy,"Today, to help protect people everywhere, the ...",eu_timeline
073e22ec2b44a5f2f6289f09dc954ecd6ca6cbb4a4b33ffd88c3c150572569c0,2020-04-15,European roadmap shows path towards common lif...,"Today, the Commission, in cooperation with the...",eu_timeline
0a3650bef3282d09d971dd92dc01570b7ecbdb67a4bdd0ff1c8cf8443a06b937,2020-12-18,Commission puts forward rules on rapid antigen...,NaN,eu_timeline
...,...,...,...,...
fa793c764aaf89c283f747036c9d73fd66f0a7e3692be56b8ac985610bef2417,2021-06-29,Commission identifies five promising candidate...,The EU Strategy on COVID-19 Therapeutics deliv...,eu_timeline
fb9c4d1634da94c73fd529b44ec40057c28a5c925a8b8e71d73c49e5a815784d,2020-03-13,Setting out coordinated response to counter th...,COVID-19 is a severe public health emergency f...,eu_timeline
fbad1d421cf80d3e0e4c6b6d3d60a2a6ec8bed4ee19e07cd15d963fa74352420,2021-01-28,Prolonging State aid Temporary Framework to fu...,The European Commission has adopted a,eu_timeline


In [19]:
# Print dataframe columns
ir_timeline_df.columns



Index(['keyword', 'page_type', 'page_link', 'department_data',
       'published_date', 'updated_date', 'title', 'content', 'content_links',
       'campaigns_links', 'part_of_links', 'documents'],
      dtype='object')

In [20]:
#What are the columns that shows dates
date_columns = [col for col in ir_timeline_df.columns if "date" in col]
ir_timeline_df[date_columns]



,published_date,updated_date
_id,,
41c0f398402f522980ec6e3ae49ef3b60b7139c1deb0ba75db830e77f62d5bdd,2021-03-24,2021-03-24
41d85d20ea2ffeaba4c27e36def4c03986ad83679fb62a07de6214abf0309ae1,2020-07-30,2020-07-30
420468c03e6550d8111344aa4d28d7e7cfd70037b16f8cab3451ff52d6bbd7b5,2020-06-06,2020-06-06
42081c8a3c63d62e4b0ef27f6a3386848b45d71665318ef14642649bb9b11b3e,2021-07-21,2021-07-23
421dd88edfd80b1fbc6c350565455e1a218774c9d4f1d74bbe2047d972187736,2020-12-10,2020-12-11
...,...,...
ff4ea2ef933287c4125d898245e878ae9fcb1b356b3c60de288dd97020bdd3a5,2021-01-22,2021-01-23
ff7508dca9aaf53565b08dc6826102bff9176584afc245b9947c18dae946506a,2021-08-25,2021-08-27
ff82338ae7c8129669562babc1c0d7a893a9c59f6d2bd00cefe853539721367e,2020-03-20,2020-03-20


In [21]:
# Create new dataframe with the wanted content and columns from Ireland timeline dataset
tmp_ir_timeline_df = pd.DataFrame(ir_timeline_df[["published_date", "title", "content"]])
tmp_ir_timeline_df["doc_source"] = "ireland_timeline"

In [22]:
# See the newly created Ireland timeline  dataframe
tmp_ir_timeline_df


,published_date,title,content,doc_source
_id,,,,
41c0f398402f522980ec6e3ae49ef3b60b7139c1deb0ba75db830e77f62d5bdd,2021-03-24,Statement by Minister for Education Norma Fole...,The State Examinations Commission is today iss...,ireland_timeline
41d85d20ea2ffeaba4c27e36def4c03986ad83679fb62a07de6214abf0309ae1,2020-07-30,From pencil and paper to drones and satellites...,July 31 2020 marks the 175th anniversary of th...,ireland_timeline
420468c03e6550d8111344aa4d28d7e7cfd70037b16f8cab3451ff52d6bbd7b5,2020-06-06,Return to Sport Expert Group recommends furthe...,"The Minister for Transport, Tourism and Sport,...",ireland_timeline
42081c8a3c63d62e4b0ef27f6a3386848b45d71665318ef14642649bb9b11b3e,2021-07-21,Government to put “special focus” on Drogheda ...,Government agrees that funding applications un...,ireland_timeline
421dd88edfd80b1fbc6c350565455e1a218774c9d4f1d74bbe2047d972187736,2020-12-10,Statement from the National Public Health Emer...,1. Hospital statistics 2. Gender of patients 3...,ireland_timeline
...,...,...,...,...
ff4ea2ef933287c4125d898245e878ae9fcb1b356b3c60de288dd97020bdd3a5,2021-01-22,Statement from the National Public Health Emer...,The Health Protection Surveillance Centre has ...,ireland_timeline
ff7508dca9aaf53565b08dc6826102bff9176584afc245b9947c18dae946506a,2021-08-25,Ministers McConalogue and Heydon launch Code o...,"The Minister for Agriculture, Food and the Mar...",ireland_timeline
ff82338ae7c8129669562babc1c0d7a893a9c59f6d2bd00cefe853539721367e,2020-03-20,Press Release on Civil Defence in the context ...,Civil Defence (Cosaint Shibhialta) is a statut...,ireland_timeline


In [23]:
def replace_non_english_content(text):
    if text is not None:
        language = detect_langs(text)
        language_details = str(language[0]).split(":")
        if language_details[0] == "en" and float(language_details[1]) > 0.95:
            return text
        else:
            return None




In [24]:
#Renaming columns in dataframes and dropping rows without content, title and date
data_frames = [tmp_cellar_df, tmp_eu_timeline_df, tmp_ir_timeline_df, tmp_pwdb_df]
for data_frame in data_frames:
    data_frame.columns = ["Date", "Title", "Content", "Document_source"]
    data_frame["Content"] = data_frame["Content"].apply(lambda x: x if x != "" else None)
    data_frame["Content"] = data_frame["Content"].replace({np.nan: None}).apply(lambda x: replace_non_english_content(x))
    data_frame.dropna(subset=['Content', 'Title', 'Date'], how="any", inplace=True)
    # data_frame.dropna(subset=['Title'], inplace=True)
    # data_frame.dropna(subset=['Date'], inplace=True)




In [25]:
#Unified datasets dataframe
unified_datasets_df = pd.DataFrame(pd.concat(data_frames))
unified_datasets_df




,Date,Title,Content,Document_source
_id,,,,
0e3d35dcad11d1d80f8fb1c35599be27fb3e864874ab1ac679fcd004723bc2e3,2020-01-30,Prior notification of a concentration (Case M....,30.1.2020 EN Official Journal of the European ...,eu_cellar
39dac43e4fffa3fc60906f3563e4d1547aea4a0f94e17cd28ec144b117e25df6,2021-08-30,COMMISSION STAFF WORKING DOCUMENT […] Accompan...,"EUROPEAN COMMISSION Brussels, 30.8.2021 SWD(20...",eu_cellar
00f65d205b8df943ae8acbd34b729c3da11da1646e7e324a6c517f95f950a5d9,2020-06-18,P9_TA(2020)0157 Amending Regulations (EU) No 5...,8.9.2021 EN Official Journal of the European U...,eu_cellar
002086bd15c9aba8b8b3cdf88498e25735cf66043ddab8877aca317256615aff,2020-09-07,Opinion No 6/2020 (pursuant to Article 287(4) ...,20.10.2020 EN Official Journal of the European...,eu_cellar
014589ceb95203c13ee4bd097ddf2f164657cb6aeb1650db280b2d8c7a57977d,2021-05-20,Council Recommendation (EU) 2021/816 20 May 20...,21.5.2021 EN Official Journal of the European ...,eu_cellar
...,...,...,...,...
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f,2020-04-12,Government support to assist the sports sector...,"As sport events have been prohibited, the fina...",pwdb
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6,2020-10-23,National public sector collective agreement 2021,"On 16 October 2020, the Government of the Repu...",pwdb
tika/3df93a8fb254187e8783d19760b465a98d88863e59a513b35a4d8062248e4190,2020-04-16,"Lifting of overtime work restrictions, extensi...",The government with a Legislative Act (Offici...,pwdb


In [27]:
#Upload to elastic
store_registry.es_index_store().put_dataframe(index_name=config.UNIFIED_DATASET_ELASTIC_SEARCH_INDEX_NAME,
                                              content=unified_datasets_df)


 98% (6116 of 6194) |################### | Elapsed Time: 0:00:00 ETA:   0:00:00

6194